# Где собирать логи
1. Ставим Docker desktop
2. Устанавливаем [образ](https://hub.docker.com/r/yandex/clickhouse-server/) Clickhouse
```
!docker run -d -p 0.0.0.0:8123:8123 --volume=/path/to/some/folder/on/disk/some_clickhouse_database:/var/lib/clickhouse --name some-clickhouse-server --ulimit nofile=262144:262144 yandex/clickhouse-server
```

Опция -p 0.0.0.0:8123:8123 открывает доступ к контейнеру по порту 8123 (иногда сразу его нет).

При повторной установке в случае ошибки вида
```
docker: Error response from daemon: Conflict. The container name "/some-clickhouse-server" is already in use by container "34899ff1c1d78111048b762fb730963adac0b90eedb9751f4c5d62aa4d90c589". You have to remove (or rename) that container to be able to reuse that name.
```
удалите контейнер командой (только замените ID контейнера на свой)
```
!docker rm 34899ff1c1d78111048b762fb730963adac0b90eedb9751f4c5d62aa4d90c589
```

Как узнать ID_контейнера
```
!docker ps
```

Как зайти в контейнер (лучше делать в командной строке):
```
docker exec -it ID_контейнера bash
```

Открыть clickhouse-client:
```
docker run -it --rm --link some-clickhouse-server:clickhouse-server yandex/clickhouse-client --host clickhouse-server
```

3. Проверьте наличие доступа к clickhouse в контейнере в браузере, открыв ссылку [localhost:8123](http://localhost:8123), должны увидеть Ok.

4. Берем открытые данные [Метрики](https://clickhouse.tech/docs/ru/getting-started/example-datasets/metrica/).

In [ ]:
!docker pull postgres

In [1]:
import requests
from io import StringIO
import pandas as pd

In [2]:
HOST = 'http://localhost:8123'

In [3]:
def clickhouse_data(query, host=HOST, timeout=60, columns=None):
    r = requests.post(host, params = {'timeout_before_checking_execution_speed': 120, 'max_execution_time': 6000}
                          , timeout = timeout, data = query)
    if r.status_code == 200:
        return r.text
    else:
        print('Что-то пошло не так')
        raise ValueError(r.text)

In [4]:
clickhouse_data('select count(*) from datasets.hits_v1')

'40065653\n'

In [5]:
text = clickhouse_data('select BrowserCountry, count(*) as cnt from datasets.hits_v1 group by BrowserCountry order by cnt desc limit 5')
text

'��\t27311685\nTp\t9652403\nTi\t1602144\nI7\t807476\nIP\t145466\n'

In [6]:
df = pd.read_csv(StringIO(text), sep='\t', names=['BrowserCountry', 'cnt'])
df

,BrowserCountry,cnt
0,��,27311685
1,Tp,9652403
2,Ti,1602144
3,I7,807476
4,IP,145466


In [7]:
%%time

text = clickhouse_data('select EventDate, count(*) from datasets.hits_v1 group by EventDate order by EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['EventDate', 'hits'])

CPU times: user 3.72 ms, sys: 1.34 ms, total: 5.07 ms
Wall time: 87.4 ms


In [8]:
df

,EventDate,hits
0,2014-03-17,6319544
1,2014-03-18,6194055
2,2014-03-19,6306972
3,2014-03-20,6079328
4,2014-03-21,5749616
5,2014-03-22,4684006
6,2014-03-23,4732132


In [9]:
%%time

text = clickhouse_data('select EventDate, uniqExact(UserID) from datasets.hits_v1 group by EventDate order by EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['EventDate', 'unique_users_exact'])
df

CPU times: user 4.13 ms, sys: 2.02 ms, total: 6.15 ms
Wall time: 257 ms


,EventDate,unique_users_exact
0,2014-03-17,36613
1,2014-03-18,36531
2,2014-03-19,36940
3,2014-03-20,36462
4,2014-03-21,35447
5,2014-03-22,31555
6,2014-03-23,31200


# pandahouse

In [10]:
%%time
text = clickhouse_data('select TraficSourceID, EventDate, uniqExact(UserID) from datasets.hits_v1 group by TraficSourceID, EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['TraficSourceID', 'EventDate', 'unique_users_exact'])

CPU times: user 5.95 ms, sys: 2.93 ms, total: 8.89 ms
Wall time: 419 ms


In [11]:
df

,TraficSourceID,EventDate,unique_users_exact
0,4,2014-03-21,528
1,6,2014-03-20,1294
2,2,2014-03-18,1606
3,-1,2014-03-21,21386
4,10,2014-03-21,3824
...,...,...,...
65,-1,2014-03-23,18574
66,10,2014-03-23,3434
67,6,2014-03-17,1410
68,0,2014-03-22,26854
